In [1]:
import os
import json
import pandas as pd

from glob import glob
from tqdm import tqdm
from collections import defaultdict
from urllib.parse import urlparse, parse_qs


In [2]:
img_dir = "/data3/htd_image20240930/htd_image"

In [3]:
df = pd.read_csv("/data3/htd_image20240930/csv/input20240930.csv")

In [4]:
df = df[df["푸드렌즈 AI 분석 내용"].notnull()]

In [5]:
def get_key(img_url):
    # parsed_url = urlparse(img_url)
    # query_params = parse_qs(parsed_url.query)
    # return query_params.get('key', [None])[0]
    key_start = img_url.find("key=")+len('key=')
    key = img_url[key_start:]
    return key

In [6]:
img_paths = glob(os.path.join(img_dir, "*.jpg"))
img_path_dict = {os.path.basename(path).split(".")[0]:path for path in img_paths}

In [7]:
import cv2

In [8]:
def convert_to_yolo_format(xmin, ymin, xmax, ymax, img_width, img_height):
    # 중심 좌표 계산
    center_x = (xmin + xmax) / 2.0
    center_y = (ymin + ymax) / 2.0
    
    # 너비와 높이 계산
    width = xmax - xmin
    height = ymax - ymin
    
    # 이미지 크기로 정규화
    center_x /= img_width
    center_y /= img_height
    width /= img_width
    height /= img_height
    
    return [center_x, center_y, width, height]

In [ ]:
data_dict = {}

for _, row in tqdm(df.iterrows(), total=len(df)):
    img_url = row["음식 사진 URL"]
    img_key = get_key(img_url)
    if img_key not in img_path_dict:
        continue
    img_path = img_path_dict[img_key]
    # print(img_path)
    img = cv2.imread(img_path)
    height, width, _ = img.shape
    
    results = row["푸드렌즈 AI 분석 내용"]
    converted_result_dict = json.loads(results.replace("'", '"'))
    
    yolo_labels = []
    
    for inner_value in converted_result_dict["foodPositionList"]:
        # get bbox data
        image_position = inner_value["imagePosition"]
        bbox = [image_position["xmin"], image_position["ymin"], image_position["xmax"], image_position["ymax"]]
        yolo_box = convert_to_yolo_format(*bbox, img_width=width, img_height=height)

        # label
        food_lens_top1 = inner_value["foodCandidates"][0]
        label = 0 if food_lens_top1['manufacturer'] in ["", "-"] else 1
        
        yolo_labels.append(", ".join([str(x) for x in [label]+yolo_box]))
    # print(yolo_labels)
    data_dict[img_path] = yolo_labels

  0%|          | 0/70643 [00:00<?, ?it/s]

  5%|▌         | 3741/70643 [18:43<2:57:32,  6.28it/s]  

dict_keys(['eatType', 'foodPositionList', 'mealType', 'predictedImagePath', 'version'])